# Bayfield Jail - Normal Tariff

- API structure: https://developer.nrel.gov/api/reopt/stable/help?API_KEY=DEMO_KEY
- Models.py: https://github.com/NREL/REopt_API/blob/master/reoptjl/models.py

In [2]:
from math import nan
import numpy as np
import pandas as pd

from scipy.optimize import minimize, LinearConstraint

from reopt_interface import setup, create_post, run_reopt, parse_dispatch_series, \
    calc_retail_electric_cost, plotly_stacked, calc_monthly_peaks2, calc_mae2, build_price_vectors

# to silence warnigns
# InsecureRequestWarning: Unverified HTTPS request is being made to host 'developer.nrel.gov'. Adding certificate verification is strongly advised.
import urllib3
urllib3.disable_warnings()

pd.options.plotting.backend='plotly'

with open('api.key', 'r') as f:
    API_KEY = f.read()
    
cfg,tariff,load,forecasts = setup('bayfield_jail.yaml')

cfg.outdir

'outputs/bayfield_jail_v30_nem_24.05.27-06.25/'

In [3]:
class dotdict(dict):
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

def costs_from_api_response(api_response):
    energy_cost =      api_response["outputs"]["ElectricTariff"]["year_one_energy_cost_before_tax"]
    demand_cost =      api_response["outputs"]["ElectricTariff"]["year_one_demand_cost_before_tax"]
    energy_revenue =   api_response["outputs"]["ElectricTariff"]["year_one_export_benefit_before_tax"]
    netcost =   energy_cost + demand_cost - energy_revenue
    
    costs = dotdict({'energy':energy_cost,
                     'demand':demand_cost,
                     'sellback':energy_revenue,
                     'total':netcost})
    
    return costs

def reopt_calc_retail_electric_cost(dispatch,cfg):
    dispatch = dispatch.copy()
    dispatch['load_and_batt_charge'] = dispatch['load'] - dispatch['batt'].clip(upper=0)
    dispatch['pv_and_batt_discharge'] = dispatch['pv'] + dispatch['batt'].clip(lower=0)

    post = create_post(cfg,
                    load_kw_series=dispatch['load_and_batt_charge'],
                    solar_cf_series=dispatch['pv_and_batt_discharge']/cfg.solar_capacity_kw,
                    batt_kw=0,
                    batt_h=0)

    api_response = run_reopt(post,api_key=API_KEY,print_results=True)
    
    return costs_from_api_response(api_response)

# Make 2019 nem prices

In [4]:
nem_prices = pd.read_csv('data/tariff/PG&E NBT EEC Values 2024 Vintage.csv',
                         index_col=0,
                         parse_dates=True,
                         comment='#')
nem_prices['True Datetime'] = pd.to_datetime(nem_prices['True Datetime'])
nem_prices['weekday'] = nem_prices.index.weekday

nem_prices.index = nem_prices['True Datetime']
nem_prices = nem_prices.sort_index()
nem_prices['true weekday'] = nem_prices.index.weekday

# uncommment to save
#nem_prices[['Price']].to_csv('data/tariff/PG&E NBT EEC Values 2024 Vintage - fake 2019.csv')

# Create Forecasts

In [5]:
forecasts = load.copy(deep=True)#pd.read_csv('data/load/2019_JPL_v4.csv',index_col=0,parse_dates=True)

maxload = forecasts[cfg.load_col].max()

forecasts['Perfect'] = forecasts[cfg.load_col]
print('Perfect',calc_mae2(forecasts['Perfect'],forecasts['Perfect'],normalize=True))

forecasts['Persist7d'] = forecasts[cfg.load_persist_col]
print('Persist7d',calc_mae2(forecasts['Perfect'],forecasts['Persist7d'],normalize=True))

# nmae 5%

forecasts['ConstantDn'] = forecasts['Perfect'] - 0.1 * forecasts['Perfect'].max()
print('ConstantDn',calc_mae2(forecasts['Perfect'],forecasts['ConstantDn'],normalize=True))

forecasts['ConstantUp'] = forecasts['Perfect'] + 0.1 * forecasts['Perfect'].max()
print('ConstantUp',calc_mae2(forecasts['Perfect'],forecasts['ConstantUp'],normalize=True))


forecasts['ScaleLinDn'] = forecasts['Perfect'] - forecasts['Perfect'] * 0.2254007
forecasts['ScaleLinUp'] = forecasts['Perfect'] + forecasts['Perfect'] * 0.2254007
print('ScaleLinDn',calc_mae2(forecasts['Perfect'],forecasts['ScaleLinDn'],normalize=True))
print('ScaleLinUp',calc_mae2(forecasts['Perfect'],forecasts['ScaleLinUp'],normalize=True))

forecasts['ScaleQuadDn'] = forecasts['Perfect'] - forecasts['Perfect']**2 * 0.47107905/maxload
forecasts['ScaleQuadUp'] = forecasts['Perfect'] + forecasts['Perfect']**2 * 0.47107905/maxload
print('ScaleQuadDn',calc_mae2(forecasts['Perfect'],forecasts['ScaleQuadDn'],normalize=True))
print('ScaleQuadUp',calc_mae2(forecasts['Perfect'],forecasts['ScaleQuadUp'],normalize=True))

# tou_levels = np.array([tariff['energyweekdayschedule'][M-1][h] for M,h in zip(forecasts.index.month,forecasts.index.hour)])
# forecasts['TouScaleDn'] = forecasts.Perfect.values - maxload*0.0333 * tou_levels
# forecasts['TouScaleUp'] = forecasts.Perfect.values + maxload*0.0333 * tou_levels
# print('TouScaleDn',calc_mae2(forecasts['Perfect'],forecasts['TouScaleDn'],normalize=True))
# print('TouScaleUp',calc_mae2(forecasts['Perfect'],forecasts['TouScaleUp'],normalize=True))

energy_prices = build_price_vectors(load.index,tariff,cfg).energy.values
forecasts['EnergyPricesDn'] = forecasts['Perfect'] - energy_prices * (1.28408091 * maxload)
forecasts['EnergyPricesUp'] = forecasts['Perfect'] + energy_prices * (1.28408091 * maxload)
print('EnergyPricesDn',calc_mae2(forecasts['Perfect'],forecasts['EnergyPricesDn'],normalize=True))
print('EnergyPricesUp',calc_mae2(forecasts['Perfect'],forecasts['EnergyPricesUp'],normalize=True))

demand_prices = build_price_vectors(load.index,tariff,cfg).demand.values
forecasts['DemandPricesDn'] = forecasts['Perfect'] - demand_prices * (0.02796934 * maxload)
forecasts['DemandPricesUp'] = forecasts['Perfect'] + demand_prices * (0.02796934 * maxload)
print('DemandPricesDn',calc_mae2(forecasts['Perfect'],forecasts['DemandPricesDn'],normalize=True))
print('DemandPricesUp',calc_mae2(forecasts['Perfect'],forecasts['DemandPricesUp'],normalize=True))

std=forecasts['Perfect'].std()
forecasts['Random'] = forecasts['Perfect'] + (13.47912488)*np.random.normal(0,1,len(forecasts))
print('Random',calc_mae2(forecasts['Perfect'],forecasts['Random'],normalize=True))

for col in forecasts.columns:
    forecasts[col] = forecasts[col].clip(lower=0)


forecasts = forecasts.drop([cfg.load_col,cfg.load_persist_col],axis=1)

#forecast.to_csv('data/load/2019_JPL_v4_fake_forecast.csv')
#forecasts.to_csv('data/load/Jail_load_solar_1min_2021-4-21_2022-4-19_profilized_fake_forecast.csv')

#forecasts = forecasts.columns

Perfect 0.0
Persist7d 0.03364146216878464
ConstantDn 0.1
ConstantUp 0.1
ScaleLinDn 0.1000231186559557
ScaleLinUp 0.10002311865595573
ScaleQuadDn 0.10003049806316532
ScaleQuadUp 0.10003049806316532
EnergyPricesDn 0.0999999996349315
EnergyPricesUp 0.0999999996349315
DemandPricesDn 0.09999996904109588
DemandPricesUp 0.0999999690410959
Random 0.10068477347524116


### Choose coefficients

In [6]:
target_nmae = 0.10#*maxload

d = load[cfg.load_col]/maxload
r = np.random.normal(0,1,len(forecasts))/maxload

s = np.random.normal(0, 1, 8760)
def forecast(x):
    #return d + x[0]
    #return d + d * x[0]
    #return d + d**2 * x[0]
    #return d + x[0]*tou_levels
    #return d + x[0]*energy_prices
    #return d + x[0]*demand_prices
    return d+r*x[0]

def obj(x):
    return (np.mean(np.abs(d-forecast(x))) - target_nmae)**2

linear_constraint = LinearConstraint([[1]], [0], [100])

r = minimize(obj,np.array([1.1]),options={'disp':True},constraints=[linear_constraint])

print('Optimal values of x:',r.x)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.6451029568232724e-17
            Iterations: 8
            Function evaluations: 16
            Gradient evaluations: 8
Optimal values of x: [13.66856706]


# Perfect

In [7]:
post = create_post(cfg)

api_response = run_reopt(post,api_key=API_KEY,print_results=True)
dispatch_perfect = parse_dispatch_series(api_response)

costs_perfect = costs_from_api_response(api_response)
#costs_perfect = calc_retail_electric_cost(dispatch_perfect,tariff,cfg)

Trying API request: 1
Polling try 1
Polling try 2
Polling try 3
Success!
Status =  optimal
Energy cost ($) =  22045.51
Demand cost ($) =  4609.41
Fixed cost ($) =  0.0
Min charge adder ($) =  0.0
Energy revenue ($) =  422.0
Net cost ($) =  26232.92

PV Size (kW) =  130.0
Storage Size (kW-kwh) =  100.0 - 200.0


In [24]:
def calc_demand_charge(month:int,prices:pd.DataFrame,tariff:dict,load:pd.Series) -> float:
    if load.name is None:
        load.name = 'P(kW)'
    demand_rate_structure = [x[0]['rate'] for x in tariff['demandratestructure']]
    nonzero_demand_rate_levels = indices_not_equal_to_value(demand_rate_structure,0)
    
    demand_charge = 0
    for tou_level in nonzero_demand_rate_levels:    
        tou_hours = indices_equal_to_value(tariff['demandweekdayschedule'][month-1],tou_level)
        monthly_peaks = calc_monthly_peaks2(load,tou_hours)

        if monthly_peaks is not None:
            demand_rate = prices.loc[monthly_peaks[f'{load.name} t'][month]].demand
            demand = monthly_peaks[f'{load.name} kw'][month]
        else: # this tou_level is not in effect for this month
            demand_rate = 0
            demand = 0

        demand_charge += demand * demand_rate
        
    return demand_charge


def calc_demand_charge2(dispatch:pd.DataFrame,tariff:dict,months:int) -> float:
    
    if isinstance(months,int):
        months = [months]

    demand_costs = []
    
    for month in months:

        price_structure=[x[0]['rate'] for x in tariff['demandratestructure']]

        df = dispatch_perfect.copy()

        dfm = df[df.index.month==month]

        c_demand = 0
        for tou_level,demand_price in enumerate(price_structure):
        #tou_level=0
        #demand_price=price_structure[tou_level]

            tou_hours_weekday = [i for i,x in enumerate(tariff['demandweekdayschedule'][month-1]) if x == tou_level]
            tou_hours_weekend = [i for i,x in enumerate(tariff['demandweekendschedule'][month-1]) if x == tou_level]

            if len(tou_hours_weekday) == 0:
                weekday_demand = 0
            else:
                dfm_wd = dfm.loc[dfm.index.weekday.isin([0,1,2,3,4])]    
                weekday_demand = dfm_wd.loc[dfm_wd.index.hour.isin(tou_hours_weekday),'grid'].max()
            if len(tou_hours_weekend) == 0:
                weekend_demand = 0
            else:
                dfm_we = dfm.loc[dfm.index.weekday.isin([5,6])]
                weekend_demand = dfm_we.loc[dfm_we.index.hour.isin(tou_hours_weekend),'grid'].max()
            demand = max(weekday_demand,weekend_demand)

            #t_demand = dfm_wd.loc[dfm_wd.index.hour.isin(tou_hours_weekday),'grid'].idxmax()

            c_demand += round(demand * demand_price,2)
            
        demand_costs.append(c_demand)

    return(demand_costs)


sum(calc_demand_charge2(dispatch_perfect,tariff,list(range(1,13))))

4609.469999999999

In [25]:
tou_hours_weekend

[]

# Solar Only

In [7]:
post = create_post(cfg,batt_kw=0,batt_h=0)

api_response = run_reopt(post,api_key=API_KEY,print_results=True)
dispatch_solar = parse_dispatch_series(api_response)

costs_solar = costs_from_api_response(api_response)
#costs_solar = calc_retail_electric_cost(dispatch_solar,tariff,cfg)

Trying API request: 1
Polling try 1
Polling try 2
Polling try 3
Success!
Status =  optimal
Energy cost ($) =  24546.16
Demand cost ($) =  7951.44
Fixed cost ($) =  0.0
Min charge adder ($) =  0.0
Energy revenue ($) =  476.0
Net cost ($) =  32021.6

PV Size (kW) =  130.0


# Forecast

In [8]:
fcast1 = 'ScaleQuadUp'
post = create_post(cfg,forecasts[fcast1])

api_response = run_reopt(post,api_key=API_KEY,print_results=True)
dispatch_forecast1 = parse_dispatch_series(api_response)

dispatch1 = dispatch_forecast1.copy()

dispatch1['load_forecast'] = dispatch1['load']
dispatch1['load'] = load[cfg.load_col]
dispatch1['grid'] = dispatch1.load - dispatch1.pv - dispatch1.batt

reopt_calc_retail_electric_cost(dispatch1,cfg)
#calc_retail_electric_cost(dispatch1,tariff,cfg)

Trying API request: 1
Polling try 1
Polling try 2
Polling try 3
Success!
Status =  optimal
Energy cost ($) =  29762.19
Demand cost ($) =  6902.24
Fixed cost ($) =  0.0
Min charge adder ($) =  0.0
Energy revenue ($) =  0.0
Net cost ($) =  36664.43

PV Size (kW) =  130.0
Storage Size (kW-kwh) =  100.0 - 200.0
Trying API request: 1
Polling try 1
Polling try 2
Polling try 3
Success!
Status =  optimal
Energy cost ($) =  22742.46
Demand cost ($) =  5425.37
Fixed cost ($) =  0.0
Min charge adder ($) =  0.0
Energy revenue ($) =  530.0
Net cost ($) =  27637.829999999998

PV Size (kW) =  130.0


{'energy': 22742.46,
 'demand': 5425.37,
 'sellback': 530.0,
 'total': 27637.829999999998}

In [9]:
fcast2 = 'Random'
post = create_post(cfg,forecasts[fcast1])

api_response = run_reopt(post,api_key=API_KEY,print_results=True)
dispatch_forecast2 = parse_dispatch_series(api_response)

dispatch2 = dispatch_forecast2.copy()

dispatch2['load_forecast'] = dispatch2['load']
dispatch2['load'] = load[cfg.load_col]
dispatch2['grid'] = dispatch2.load - dispatch2.pv - dispatch2.batt

reopt_calc_retail_electric_cost(dispatch2,cfg)
#calc_retail_electric_cost(dispatch1,tariff,cfg)

Trying API request: 1
Polling try 1
Polling try 2
Polling try 3
Success!
Status =  optimal
Energy cost ($) =  29762.19
Demand cost ($) =  6902.24
Fixed cost ($) =  0.0
Min charge adder ($) =  0.0
Energy revenue ($) =  0.0
Net cost ($) =  36664.43

PV Size (kW) =  130.0
Storage Size (kW-kwh) =  100.0 - 200.0
Trying API request: 1
Polling try 1
Polling try 2
Polling try 3
Success!
Status =  optimal
Energy cost ($) =  22742.46
Demand cost ($) =  5425.37
Fixed cost ($) =  0.0
Min charge adder ($) =  0.0
Energy revenue ($) =  530.0
Net cost ($) =  27637.829999999998

PV Size (kW) =  130.0


{'energy': 22742.46,
 'demand': 5425.37,
 'sellback': 530.0,
 'total': 27637.829999999998}

# Plot

## Actual

In [10]:
month = 9
peak_hours = list(range(9,21))

In [11]:
monthly_peaks = calc_monthly_peaks2(dispatch1.grid,peak_hours)
if monthly_peaks is not None:
    peak = monthly_peaks.iloc[month-1]['grid kw']
    t_peak = monthly_peaks.iloc[month-1]['grid t']
else:
    peak = None
    t_peak = None   

begin = t_peak - pd.Timedelta('1h')*36
end = t_peak + pd.Timedelta('1h')*36

f = plotly_stacked(dispatch1.loc[f'2019-{month}'],#[begin:end],
                    solar='pv',
                    load='load',
                    load_forecast='load_forecast',
                    batt='batt',
                    utility='grid',
                    soc='soc',
                    theme='plotly_dark',
                    title=f'{fcast1} Forecast<br>Monthly Peak {peak} kW at {t_peak}')

In [12]:
monthly_peaks = calc_monthly_peaks2(dispatch2.grid,peak_hours)
if monthly_peaks is not None:
    peak2 = monthly_peaks.iloc[month-1]['grid kw']
    t_peak2 = monthly_peaks.iloc[month-1]['grid t']
else:
    peak2 = None
    t_peak2 = None   

begin2 = t_peak2 - pd.Timedelta('1h')*36
end2 = t_peak2 + pd.Timedelta('1h')*36

f = plotly_stacked(dispatch2.loc[f'2019-{month}'],#[begin:end],
                    solar='pv',
                    load='load',
                    load_forecast='load_forecast',
                    batt='batt',
                    utility='grid',
                    soc='soc',
                    theme='plotly_dark',
                    title=f'{fcast2} Forecast<br>Monthly Peak {peak2} kW at {t_peak2}')

## Perfect

In [13]:
load_at_peak = dispatch_perfect.loc[t_peak,'grid']

f = plotly_stacked(dispatch_perfect.loc[f'2019-{month}'],#[begin:end],
                    solar='pv',
                    load='load',
                    load_forecast='load_forecast',
                    batt='batt',
                    utility='grid',
                    soc='soc',
                    theme='plotly_dark',
                    title=f'Load {load_at_peak:.1f} kW at {t_peak}')

# Multiple Forecasts

Set up results df

In [14]:
# Ce cost of energy, Cd cost of demand, Re revenue of energy, Ct total cost
# p_x is performance based on Ce, Cd, Re, or Ct
# pk_x peak of month x, tpk_x time of peak of month x
cols = ['Ce','Cd','Re','Ct','p_Ce','p_Cd','p_Re','p_Ct']
cols = cols + [f'pk_{m}' for m in range(1,13)] + [f'tpk_{m}' for m in range(1,13)]
cols = cols + ['dispatch']
r = pd.DataFrame(columns=cols,index=forecasts.columns)

Insert perfect results

In [15]:
peaks = calc_monthly_peaks2(dispatch_perfect.grid,peak_hours)['grid kw'].values
t_peaks = calc_monthly_peaks2(dispatch_perfect.grid,peak_hours)['grid t'].values

r.loc['Perfect'] = {'Ce':costs_perfect.energy,'Cd':costs_perfect.demand,'Re':costs_perfect.sellback,'Ct':costs_perfect.total,
                    'p_Ce':1,'p_Cd':1,'p_Re':1,'p_Ct':1,
                    'pk_1':peaks[0],'pk_2':peaks[1],'pk_3':peaks[2],'pk_4':peaks[3],'pk_5':peaks[4],'pk_6':peaks[5],'pk_7':peaks[6],'pk_8':peaks[7],'pk_9':peaks[8],'pk_10':peaks[9],'pk_11':peaks[10],'pk_12':peaks[11],
                    'tpk_1':t_peaks[0],'tpk_2':t_peaks[1],'tpk_3':t_peaks[2],'tpk_4':t_peaks[3],'tpk_5':t_peaks[4],'tpk_6':t_peaks[5],'tpk_7':t_peaks[6],'tpk_8':t_peaks[7],'tpk_9':t_peaks[8],'tpk_10':t_peaks[9],'tpk_11':t_peaks[10],'tpk_12':t_peaks[11],
                    'dispatch':dispatch_perfect}

Insert solar-only results

In [16]:
peaks =   calc_monthly_peaks2(dispatch_solar.grid,peak_hours)['grid kw'].values
t_peaks = calc_monthly_peaks2(dispatch_solar.grid,peak_hours)['grid t'].values

r.loc['SolarOnly'] = {'Ce':costs_solar.energy,'Cd':costs_solar.demand,'Re':costs_solar.sellback,'Ct':costs_solar.total,
                    'p_Ce':0,'p_Cd':0,'p_Re':0,'p_Ct':0,
                    'pk_1':peaks[0],'pk_2':peaks[1],'pk_3':peaks[2],'pk_4':peaks[3],'pk_5':peaks[4],'pk_6':peaks[5],'pk_7':peaks[6],'pk_8':peaks[7],'pk_9':peaks[8],'pk_10':peaks[9],'pk_11':peaks[10],'pk_12':peaks[11],
                    'tpk_1':t_peaks[0],'tpk_2':t_peaks[1],'tpk_3':t_peaks[2],'tpk_4':t_peaks[3],'tpk_5':t_peaks[4],'tpk_6':t_peaks[5],'tpk_7':t_peaks[6],'tpk_8':t_peaks[7],'tpk_9':t_peaks[8],'tpk_10':t_peaks[9],'tpk_11':t_peaks[10],'tpk_12':t_peaks[11],
                    'dispatch':dispatch_solar}

In [17]:
fcast1 = 'Persist7d'
post = create_post(cfg,forecasts[fcast1])

api_response = run_reopt(post,api_key=API_KEY,print_results=True)
dispatch_forecast1 = parse_dispatch_series(api_response)

dispatch1 = dispatch_forecast1.copy()

dispatch1['load_forecast'] = dispatch1['load']
dispatch1['load'] = load[cfg.load_col]
dispatch1['grid'] = dispatch1.load - dispatch1.pv - dispatch1.batt

reopt_calc_retail_electric_cost(dispatch1,cfg)
#calc_retail_electric_cost(dispatch1,tariff,cfg)

Trying API request: 1
Polling try 1
Polling try 2
Polling try 3
Success!
Status =  optimal
Energy cost ($) =  21904.57
Demand cost ($) =  4524.32
Fixed cost ($) =  0.0
Min charge adder ($) =  0.0
Energy revenue ($) =  6.0
Net cost ($) =  26422.89

PV Size (kW) =  130.0
Storage Size (kW-kwh) =  100.0 - 200.0
Trying API request: 1
Polling try 1
Polling try 2
Polling try 3
Success!
Status =  optimal
Energy cost ($) =  22140.92
Demand cost ($) =  6105.94
Fixed cost ($) =  0.0
Min charge adder ($) =  0.0
Energy revenue ($) =  136.0
Net cost ($) =  28110.859999999997

PV Size (kW) =  130.0


{'energy': 22140.92,
 'demand': 6105.94,
 'sellback': 136.0,
 'total': 28110.859999999997}

In [28]:
for fcast in forecasts.columns[1:]:    
    print('///// ',fcast,' /////')
    post = create_post(cfg,forecasts[fcast])
    try:
        api_response = run_reopt(post,api_key=API_KEY,print_results=True)
        d = parse_dispatch_series(api_response)
        #d = d.copy()
        
        # make the dispatch actual
        d['load_forecast'] = d['load']
        d['load'] = load[cfg.load_col]
        d['grid'] = d.load - d.pv - d.batt

        c = reopt_calc_retail_electric_cost(d,cfg)
        
        pks =   calc_monthly_peaks2(d.grid,peak_hours)['grid kw'].values
        tpks = calc_monthly_peaks2(d.grid,peak_hours)['grid t'].values
        
        p_Ce=round(1 - (c.energy-costs_perfect.energy)/(costs_solar.energy-costs_perfect.energy),3)
        if (costs_solar.demand-costs_perfect.demand) != 0:
            p_Cd=round(1 - (c.demand-costs_perfect.demand)/(costs_solar.demand-costs_perfect.demand),3)
        else:
            p_Cd=nan
        p_Re=round(1 - (c.sellback-costs_perfect.sellback)/(costs_solar.sellback-costs_perfect.sellback),3)
        p_Ct=round(1 - (c.total-costs_perfect.total)/(costs_solar.total-costs_perfect.total),3)
        
        
        r.loc[fcast] = {'Ce':c.energy,'Cd':c.demand,'Re':c.sellback,'Ct':c.total,
                        'p_Ce':p_Ce,'p_Cd':p_Cd,'p_Re':p_Re,'p_Ct':p_Ct,
                        'pk_1':pks[0],'pk_2':pks[1],'pk_3':pks[2],'pk_4':pks[3],'pk_5':pks[4],'pk_6':pks[5],'pk_7':pks[6],'pk_8':pks[7],'pk_9':pks[8],'pk_10':pks[9],'pk_11':pks[10],'pk_12':pks[11],
                        'tpk_1':tpks[0],'tpk_2':tpks[1],'tpk_3':tpks[2],'tpk_4':tpks[3],'tpk_5':tpks[4],'tpk_6':tpks[5],'tpk_7':tpks[6],'tpk_8':tpks[7],'tpk_9':tpks[8],'tpk_10':tpks[9],'tpk_11':tpks[10],'tpk_12':tpks[11],
                        'dispatch':d}
    except:
        print('API Request fully failed')
    

/////  Persist7d  /////
Trying API request: 1
Polling try 1
Polling try 2
Polling try 3
Success!
Status =  optimal
Energy cost ($) =  21904.57
Demand cost ($) =  4524.32
Fixed cost ($) =  0.0
Min charge adder ($) =  0.0
Energy revenue ($) =  6.0
Net cost ($) =  26422.89

PV Size (kW) =  130.0
Storage Size (kW-kwh) =  100.0 - 200.0
Trying API request: 1
Polling try 1
Polling try 2


In [ ]:
r

,Ce,Cd,Re,Ct,p_Ce,p_Cd,p_Re,p_Ct,pk_1,pk_2,...,tpk_4,tpk_5,tpk_6,tpk_7,tpk_8,tpk_9,tpk_10,tpk_11,tpk_12,dispatch
Perfect,21867.31,4608.50,8.00,26467.81,1,1,1,1,42.80,44.50,...,2019-04-21T20:00:00.000000000,2019-05-18T19:00:00.000000000,2019-06-01T15:00:00.000000000,2019-07-21T20:00:00.000000000,2019-08-24T15:00:00.000000000,2019-09-03T16:00:00.000000000,2019-10-06T18:00:00.000000000,2019-11-17T11:00:00.000000000,2019-12-01T11:00:00.000000000,load pv batt soc ...
Persist7d,22140.92,6105.94,136.00,28110.86,0.90,0.55,0.73,0.70,42.80,44.50,...,2019-04-21T20:00:00.000000000,2019-05-18T19:00:00.000000000,2019-06-03T10:00:00.000000000,2019-07-24T15:00:00.000000000,2019-08-06T18:00:00.000000000,2019-09-27T15:00:00.000000000,2019-10-15T16:00:00.000000000,2019-11-17T11:00:00.000000000,2019-12-01T11:00:00.000000000,load pv batt soc ...
ConstantDn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ConstantUp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ScaleLinDn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ScaleLinUp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ScaleQuadDn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ScaleQuadUp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EnergyPricesDn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EnergyPricesUp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
r[['Ce','Cd','Re','Ct']].sort_values('Ct')

,Ce,Cd,Re,Ct
Perfect,21867.31,4608.50,8.00,26467.81
Persist7d,22140.92,6105.94,136.00,28110.86
SolarOnly,24546.16,7951.44,476.00,32021.60
ConstantDn,NaN,NaN,NaN,NaN
ConstantUp,NaN,NaN,NaN,NaN
ScaleLinDn,NaN,NaN,NaN,NaN
ScaleLinUp,NaN,NaN,NaN,NaN
ScaleQuadDn,NaN,NaN,NaN,NaN
ScaleQuadUp,NaN,NaN,NaN,NaN
EnergyPricesDn,NaN,NaN,NaN,NaN


In [ ]:
r[['p_Ce','p_Cd','p_Re','p_Ct']].sort_values('p_Ct')

,p_Ce,p_Cd,p_Re,p_Ct
SolarOnly,0,0,0,0
Persist7d,0.90,0.55,0.73,0.70
Perfect,1,1,1,1
ConstantDn,NaN,NaN,NaN,NaN
ConstantUp,NaN,NaN,NaN,NaN
ScaleLinDn,NaN,NaN,NaN,NaN
ScaleLinUp,NaN,NaN,NaN,NaN
ScaleQuadDn,NaN,NaN,NaN,NaN
ScaleQuadUp,NaN,NaN,NaN,NaN
EnergyPricesDn,NaN,NaN,NaN,NaN


In [ ]:
r[['p_Ce','p_Cd','p_Re','p_Ct']].corr()['p_Ct']

p_Ce   0.98
p_Cd   0.99
p_Re   1.00
p_Ct   1.00
Name: p_Ct, dtype: float64

In [ ]:
r[['p_Ct','p_Cd','p_Ce','p_Re']].sort_values('p_Ct').plot(width=800)

In [ ]:
r.sort_values('p_Ct')[[f'pk_{i}' for i in range(1,13)]].T

,SolarOnly,Persist7d,Perfect,ConstantDn,ConstantUp,ScaleLinDn,ScaleLinUp,ScaleQuadDn,ScaleQuadUp,EnergyPricesDn,EnergyPricesUp,DemandPricesDn,DemandPricesUp,Random
pk_1,51.00,42.80,42.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pk_2,51.50,44.50,44.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pk_3,52.30,42.40,42.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pk_4,48.60,38.50,38.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pk_5,67.80,45.30,45.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pk_6,88.30,78.90,60.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pk_7,88.70,74.80,60.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pk_8,87.70,69.20,52.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pk_9,81.90,73.60,56.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pk_10,72.10,62.00,47.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
r.sort_values('p_Ct')[[f'pk_{i}' for i in range(1,13)]].T.plot(width=800)

In [ ]:
r.drop('dispatch',axis=1).to_csv(cfg.outdir+'results.csv')

In [ ]:
for fcast,df in zip(r.index,r.dispatch):
    df.to_csv(cfg.outdir+f'dispatch_{fcast}.csv')

AttributeError: 'float' object has no attribute 'to_csv'

In [ ]:
cfg

{'version': 30,
 'note': 'PUN_constant_sell_price',
 'dev': False,
 'test': False,
 'data_dir': 'data/',
 'out_dir': 'outputs/',
 'interval': '1h',
 'year': 2019,
 'load_file': 'data/load/Jail_load_solar_1min_2021-4-21_2022-4-19_profilized.csv',
 'load_col': 'Load [kW]',
 'load_persist_col': 'Load Persist 7d [kW]',
 'load_forecast_file': 'data/load/Jail_load_solar_1min_2021-4-21_2022-4-19_profilized_fake_forecast.csv',
 'load_forecast_col': 'Persist7d',
 'solar_file': 'data/load/Jail_load_solar_1min_2021-4-21_2022-4-19_profilized.csv',
 'solar_col': 'Solar [kW]',
 'solar_scaler': 130,
 'solar_kw': 130,
 'batt_kw': 100,
 'batt_h': 2,
 'tariff_file': 'data/tariff/bayfield_no_demand.json',
 'energy_price_buy_file': 'data/tariff/network_prices_1h.csv',
 'energy_price_buy_col': 'pun',
 'energy_price_sell_constant': 0.025,
 'post': {'Settings': {'time_steps_per_hour': 1},
  'Site': {'longitude': 45.50589, 'latitude': 9.16099},
  'PV': {'array_type': 0,
   'can_net_meter': False,
   'min_kw':